# Day 4: Giant Squid
## Part One
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is *marked* on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board *wins*. (Diagonals don't count.)

The submarine has a *bingo subsystem* to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:
```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```
After the first five numbers are drawn (7, 4, 9, 5, and 11), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
 ```
After the next six numbers are drawn (17, 23, 2, 0, 14, and 21), there are still no winners:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
Finally, 24 is drawn:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: `14 21 17 24 4`).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is `188`. Then, multiply that sum by the number that was just called when the board won, `24`, to get the final score, `188 * 24 = 4512`.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

### Solution
Let's leverage the power of OOP to represent a board. Let's start implementing a number on the board, which can be marked or not.

In [25]:
from dataclasses import dataclass

@dataclass
class Number:
    number: int
    marked: bool = False

Now let's represent a board.

In [86]:
from colorama import Fore, Style
from copy import copy
from itertools import product
from typing import Optional


class Board:
    SIZE = 5

    board: list[list[Number]]
    _lookup: dict[int, tuple[int, int]]

    def __init__(self, board: list[list[Number]]) -> None:
        self.board = copy(board)
        self._build_lookup()

    def _build_lookup(self) -> None:
        self._lookup = dict()
        for i, j in product(range(Board.SIZE), range(Board.SIZE)):
            self._lookup[self.board[i][j].number] = (i, j)

    def mark_number(self, number: int) -> None:
        try:
            i, j = self._lookup[number]
        except KeyError:
            return
        self.board[i][j].marked = True

    def bingo(self) -> bool:
        return self.marked_row() is not None \
            or self.marked_column() is not None

    def marked_row(self) -> Optional[list[Number]]:
        for row in self.board:
            if all(n.marked for n in row):
                return row

    def marked_column(self) -> Optional[list[Number]]:
        for c in range(Board.SIZE):
            column = (self.board[r][c] for r in range(Board.SIZE))
            if all(n.marked for n in column):
                return list(column)
    
    def __repr__(self) -> str:
        # colored = [f"{Fore.RED if num.marked else Fore.GREEN}{str(num.number).rjust(2, ' ')}{Style.RESET_ALL}" for row in self.board for num in row]
        # return " ".join([c for r in range(Board.SIZE) for c in range(Board.SIZE)])
        return "\n".join([" ".join(map(lambda n: f"{Fore.RED if n.marked else Fore.WHITE}{str(n.number).rjust(2, ' ')}{Style.RESET_ALL}", row)) for row in self.board])
    
    def __str__(self) -> str:
        return self.__repr__()


Then we can import the drawn numbers and the boards from the input file:

In [32]:
from io import TextIOWrapper


def import_drawn_numbers(file: TextIOWrapper) -> list[int]:
    return list(map(int, file.readline().strip().split(",")))


def import_boards(file: TextIOWrapper) -> list[Board]:
    board_buffer: list[list[Number]] = []
    boards: list[Board] = []
    for row in file:
        if row != "\n":
            board_buffer.append([Number(int(n)) for n in row.strip().split()])
        else:
            boards.append(Board(board_buffer))
            board_buffer.clear()
    return boards


In [93]:
with open("input.txt", "r") as file:
    drawn_numbers = import_drawn_numbers(file)
    boards = import_boards(file)

winning_board = None
last_drawn_number = 0
for number in drawn_numbers:
    last_drawn_number = number
    for board in boards:
        board.mark_number(number)
        if board.bingo():
            winning_board = board
            break
    if winning_board is not None:
        break


if winning_board is not None:
    print(winning_board)
    print(last_drawn_number * sum(num.number for row in winning_board.board for num in row if not num.marked))

63  3 22  7 10
36 76 14 77 38
48 27 40  9 60
31 56 75 74 78
86 64 71 90 67
44736


## Part Two
On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to *figure out which board will win last* and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after `13` is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to `148` for a final score of `148 * 13 = 1924`.

Figure out which board will win last. *Once it wins, what would its final score be*?

### Solution

In [92]:
with open("input.txt", "r") as file:
    drawn_numbers = import_drawn_numbers(file)
    boards = import_boards(file)

last_winning_board = None
last_winning_drawn_number = 0
for number in drawn_numbers:
    for board in boards:
        if board.bingo():
            continue
        board.mark_number(number)
        if board.bingo():
            last_winning_drawn_number = number
            last_winning_board = board


if last_winning_board is not None:
    print(last_winning_board)
    print(last_winning_drawn_number * sum(num.number for row in last_winning_board.board for num in row if not num.marked))

 1 20 23 79 14
27 76  3 90 85
88 35  7 10 92
67 97 59 41  8
56 57 65 45 81
1827
